# LangChain 核心模块学习：Model I/O

`Model I/O` 是 LangChain 为开发者提供的一套面向 LLM 的标准化模型接口，包括模型输入（Prompts）、模型输出（Output Parsers）和模型本身（Models）。

- Prompts：模板化、动态选择和管理模型输入
- Models：以通用接口调用语言模型
- Output Parser：从模型输出中提取信息，并规范化内容

![](../images/model_io.jpeg)


In [ ]:
# 安装最新版本的 LangChain Python SDK（https://github.com/langchain-ai/langchain）
!pip install -U langchain

## 输出解析器 Output Parser

**语言模型的输出是文本。**

但很多时候，您可能希望获得比纯文本**更结构化的信息**。这就是输出解析器的价值所在。

输出解析器是帮助结构化语言模型响应的类。它们必须实现两种主要方法：

- "获取格式指令"：返回一个包含有关如何格式化语言模型输出的字符串的方法。
- "解析"：接受一个字符串（假设为来自语言模型的响应），并将其解析成某种结构。

然后还有一种可选方法：
- "使用提示进行解析"：接受一个字符串（假设为来自语言模型的响应）和一个提示（假设为生成此响应的提示），并将其解析成某种结构。在需要重新尝试或修复输出，并且需要从提示中获取信息以执行此操作时，通常会提供提示。

### 列表解析 List Parser

当您想要返回一个逗号分隔的项目列表时，可以使用此输出解析器。

In [1]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI

# 创建一个输出解析器，用于处理带逗号分隔的列表输出
output_parser = CommaSeparatedListOutputParser()

# 获取格式化指令，该指令告诉模型如何格式化其输出
format_instructions = output_parser.get_format_instructions()
print(format_instructions)



In [2]:
# 创建一个提示模板，它会基于给定的模板和变量来生成提示
prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",  # 模板内容
    input_variables=["subject"],  # 输入变量
    partial_variables={"format_instructions": format_instructions}  # 预定义的变量，这里我们传入格式化指令
)

Your response should be a list of comma separated values, eg: `foo, bar, baz`


In [ ]:
# 使用提示模板和给定的主题来格式化输入
_input = prompt.format(subject="ice cream flavors")

In [ ]:
print(_input)

In [ ]:
llm = OpenAI(temperature=0)

In [ ]:
output = llm(_input)

In [ ]:
print(output)

In [ ]:
# 使用之前创建的输出解析器来解析模型的输出
output_parser.parse(output)

### 日期解析 Datatime Parser

In [ ]:
from langchain.output_parsers import DatetimeOutputParser
from langchain.chains import LLMChain

output_parser = DatetimeOutputParser()
template = """Answer the users question:

{question}

{format_instructions}"""

prompt = PromptTemplate.from_template(
    template,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

In [ ]:
print(prompt)

In [ ]:
print(prompt.format(question="around when was bitcoin founded?"))

In [ ]:
chain = LLMChain(prompt=prompt, llm=OpenAI())

In [ ]:
output = chain.run("around when was bitcoin founded?")

In [ ]:
output

In [ ]:
output_parser.parse(output)

In [ ]:
print(output_parser.parse(output))